# PUC Minas
## Pós-Graduação em Ciência de Dados e Big Data
## Machine Learning
## Professor: Gabriel Oliveira Assunção


-------------

# Lista 1
-------------

#### Alunos:
- Guilherme de Gouvêa Almada
- Pedro Henrique Batista Chitarra

-------------

Objetivo: treinar um modelo de classificação utilizando o modelo de regressão Logística e árvore de decisão.


Banco utilizado: Detecção de doenças no fígado.


[Link](https://drive.google.com/file/d/1jnLwuv4e_ZeCqluMXs3Mar2TGetumtVv/view?usp=drive_link) para o banco


Descrição do banco:
> Este banco contém registro de 416 pacientes diagnosticados com doença no fígado e 167 pacientes sem. Esta informação está na coluna Selector.
>
> Há 10 variáveis no banco:
>
> * age: idade
> * Gender: gênero do paciente
> * TB: Bilirrubina total
> * DB: Bilirrubina direta
> * Alkphos: fosfatase alcalina.
> * Sgpt: transaminase glutâmico-pirúvica sérica (TGP)
> * Sgot: transaminase glutâmico-oxalacética sérica (TGO)
> * TP: Proteína total
> * ALB: Albumina
> * A/G Ratio: Relação Albumina:Globulina


Exercício:


1. Carregue o banco de dados e analise suas features. Transforme a feature sexo em uma variavel dummy.
2. Separe o banco em 80% para treino e 20% para teste.
3. Treine um modelo de regressão logística.
   1. Interprete dois coeficientes.
4. Treine um modelo de árvore de decisão. Utilize max_depth de 4.
   1. Quais são as duas features mais importantes?
5. Construa um relatório dos dois modelos.
   1. Tomando como decisão a precisão para detectar doença no fígado qual é o melhor modelo?
   2. Tomando como decisão o f1 score do rótulo de pacientes com doença no fígado qual é o melhor modelo?


Obs.: Utilize o **mesmo** banco de treino e de teste para construir e avaliar os dois modelos.

## 1. Carga do banco
--------------------

In [87]:
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (classification_report, confusion_matrix,
                             roc_auc_score, roc_curve, RocCurveDisplay,
                             recall_score)

In [88]:
df = pd.read_csv("Indian Liver Patient Dataset (ILPD).csv")

In [89]:
df["Selector"].value_counts()

Selector
1    416
2    167
Name: count, dtype: int64

Como temos 416 pacientes diagnosticados com doença no fígado e 167 pacientes
sem, sabemos pela contagem cima que 1 indica que o paciente tem a doença e 2
indica que o paciente não tem a doença. Para fazer os modelos de classificação,
queremos trabalhar com 0 para não ter a doença e 1 para ter a doença. Por isso,
vamos converter o valor 2 para 0.

In [90]:
df["Selector"] = df["Selector"].map({2: 0, 1: 1})

In [91]:
df["Selector"].value_counts()

Selector
1    416
0    167
Name: count, dtype: int64

In [92]:
df.describe()

,Age,TB,DB,Alkphos,Sgpt,Sgot,TP,ALB,A/G Ratio,Selector
count,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,583.000000,579.000000,583.000000
mean,44.746141,3.298799,1.486106,290.576329,80.713551,109.910806,6.483190,3.141852,0.947064,0.713551
std,16.189833,6.209522,2.808498,242.937989,182.620356,288.918529,1.085451,0.795519,0.319592,0.452490
min,4.000000,0.400000,0.100000,63.000000,10.000000,10.000000,2.700000,0.900000,0.300000,0.000000
25%,33.000000,0.800000,0.200000,175.500000,23.000000,25.000000,5.800000,2.600000,0.700000,0.000000
50%,45.000000,1.000000,0.300000,208.000000,35.000000,42.000000,6.600000,3.100000,0.930000,1.000000
75%,58.000000,2.600000,1.300000,298.000000,60.500000,87.000000,7.200000,3.800000,1.100000,1.000000
max,90.000000,75.000000,19.700000,2110.000000,2000.000000,4929.000000,9.600000,5.500000,2.800000,1.000000


In [93]:
df.isna().sum()

Age          0
Gender       0
TB           0
DB           0
Alkphos      0
Sgpt         0
Sgot         0
TP           0
ALB          0
A/G Ratio    4
Selector     0
dtype: int64

Temos NAs no banco, então vamos tratar esses valores.

In [94]:
df = df.dropna(subset=["A/G Ratio"])

Convertendo a variável sexo em dummy

In [95]:
df = pd.get_dummies(df, drop_first=True)
df

,Age,TB,DB,Alkphos,Sgpt,Sgot,TP,ALB,A/G Ratio,Selector,Gender_Male
0,65,0.7,0.1,187,16,18,6.8,3.3,0.90,1,False
1,62,10.9,5.5,699,64,100,7.5,3.2,0.74,1,True
2,62,7.3,4.1,490,60,68,7.0,3.3,0.89,1,True
3,58,1.0,0.4,182,14,20,6.8,3.4,1.00,1,True
4,72,3.9,2.0,195,27,59,7.3,2.4,0.40,1,True
...,...,...,...,...,...,...,...,...,...,...,...
578,60,0.5,0.1,500,20,34,5.9,1.6,0.37,0,True
579,40,0.6,0.1,98,35,31,6.0,3.2,1.10,1,True
580,52,0.8,0.2,245,48,49,6.4,3.2,1.00,1,True
581,31,1.3,0.5,184,29,32,6.8,3.4,1.00,1,True


## 2. Quebra do banco
--------------------

In [96]:
target = "Selector"
features = df.columns.drop(target)
x = df[features]
y = df[target]

In [97]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=2506)

Visualizando a distribuição dos dados no conjunto de treino

In [98]:
fig = px.pie(df, names="Selector")
fig.update_layout(title_text='Porcentagem de pessoas com doença no fígado')
fig.show()

## 3. Primeiro modelo
--------------------

In [99]:
logistic_model = LogisticRegression(random_state=2506)
logistic_model.fit(x_train, y_train)

/home/pedro/anaconda3/envs/clubedorobson/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression(random_state=2506)

### 3.1 Interpretação de dois coeficientes
--------------------

In [100]:
coefs_df = pd.DataFrame(logistic_model.coef_.T, index=x_train.columns)
coefs_df

,0
Age,0.016980
TB,-0.267018
DB,0.942782
Alkphos,0.000717
Sgpt,0.013197
Sgot,0.001181
TP,0.220560
ALB,-0.289874
A/G Ratio,-0.504077
Gender_Male,-0.083770


### Razão de chances

#### Bilirrubina direta

In [101]:
coefs_df.loc["DB"]

0    0.942782
Name: DB, dtype: float64

In [102]:
razao_db = np.exp(coefs_df.loc["DB"].values[0])
razao_db

2.567112213689039

Mantendo as outras variáveis constantes, para cada aumento na Bilirrubina
direta, o odds de se ter doença no fígado é aproximadamente $2.56$ vezes maior.
Ou seja, quanto maior seu valor, menor a chance de ter a doença.

#### A/G Ratio

In [103]:
coefs_df.loc["A/G Ratio"]

0   -0.504077
Name: A/G Ratio, dtype: float64

In [104]:
razao_ag = np.exp(coefs_df.loc["A/G Ratio"].values[0])
razao_ag

0.6040626140833871

Mantendo as outras variáveis constantes, para cada aumento no A/G Ratio,
o odds de se ter doença no fígado é aproximadamente $0.604$ vezes menor. Ou
seja, quanto menor seu valor, menor a chance de ter a doença.

## 4. Segundo modelo
--------------------

In [105]:
model_tree = DecisionTreeClassifier(random_state=2506, max_depth=4)
model_tree.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=4, random_state=2506)

### 4.1 Duas features mais importantes
--------------------

In [106]:
feature_importance_df = pd.DataFrame(model_tree.feature_importances_.T,
                                     index=x_train.columns,
                                     columns=['Importancia'])
feature_importance_df = feature_importance_df.sort_values(
    'Importancia', ascending=False)

In [107]:
px.bar(feature_importance_df.sort_values('Importancia', ascending=False))

As duas variáveis mais importantes são a DB (Bilirrubina direta) e a
Sgpt (transaminase glutâmico-pirúvica sérica (TGP))

## 5. Avaliação do modelo
--------------------

### Regressão Logística

In [108]:
y_hat_logistic = logistic_model.predict(x_test)
print(classification_report(y_test, y_hat_logistic))

              precision    recall  f1-score   support

           0       0.41      0.33      0.37        21
           1       0.86      0.89      0.88        95

    accuracy                           0.79       116
   macro avg       0.64      0.61      0.62       116
weighted avg       0.78      0.79      0.78       116



### Árvore de Decisão

In [109]:
y_hat_tree = model_tree.predict(x_test)
print(classification_report(y_test, y_hat_tree))

              precision    recall  f1-score   support

           0       0.12      0.05      0.07        21
           1       0.81      0.93      0.87        95

    accuracy                           0.77       116
   macro avg       0.47      0.49      0.47       116
weighted avg       0.69      0.77      0.72       116



### 5.1 Qual modelo escolhido olhando para a precisão de detecção de doença no fígado?
--------------------

O valor que representa a classe com doença no fígado é 1. Portanto, para o modelo
de Regressão Logística a precisão foi de $0.86$ e para o modelo de Árvore de Decisão foi de $0.81$.

Logo, o melhor modelo olhando a precisão de detecção de doença no fígado é o modelo	
de Regressão Logística.

### 5.2 Qual modelo escolhido olhando para o f1-score do rótulo de doença do fígado?
--------------------

O valor que representa a classe com doença no fígado é 1. Portanto, para o modelo
de Regressão Logística o f1-score foi de $0.88$ e para o modelo de Árvore de Decisão foi de $0.87$.

Logo, o melhor modelo olhando o f1-score do rótulo de doença do fígado é o modelo
de Regressão Logística.